In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

In [2]:
# this function generates all the combination of states and assisgn a value to it if needs = v/ else assign a possible action
# for instance： n_agent = 1
# gets {[0,0] : 0}, {[0,1]: 0}, {[0,2]: 0}...
def getMap(maps={}, needs='a', maxA=3, cash=()):
    if len(cash) == 0:
        n = maxA - 1
        s = [(0, 0) for _ in range(maxA)]
    else:
        n = cash[0]
        s = cash[1]

    if n == 0:
        for i in range(5):
            for j in range(5):
                s[n] = (i, j)
                if needs == 'a':
                    maps[str(s)] = [0] * maxA
                else:
                    maps[str(s)] = 0
    else:
        for i in range(5):
            for j in range(5):
                s[n] = (i, j)
                getMap(maps, needs, maxA, (n - 1, s.copy()))
    return maps

In [3]:
# this is an iterrator
# for a, p in getActionandProb(action, pe)
# it will generates all the possible combination of actions given aciton and pe
# for instance, for action 0 and pe = 0.9
# we will have a = 0, p = 0.9, a = 1, p = 0.025...
class getActionandProb:
    def __init__(self, action, pe):
        self.action = action
        self.length = len(action)
        self.curr = [0] * self.length
        self.pes = [pe ** i * (0.25 - pe / 4) ** (self.length - i) for i in range(self.length + 1)]
        self.ite = 0

    def __iter__(self):
        return self

    def __next__(self):
        x = self.curr.copy()
        c = sum([i == j for i, j in zip(x, self.action)])
        if self.ite < 5 ** self.length:
            self.ite += 1
            for i in range(self.length - 1, -1, -1):
                if self.curr[i] + 1 == 5:
                    self.curr[i] = 0
                else:
                    self.curr[i] += 1
                    break
            return x, self.pes[c]
        else:
            raise StopIteration

In [4]:
# givens a state, it returns features,values and help update the theta
# here we calculates six states: number of agents in target/ in Rw / in wall / collision/ distances to each store
# you can change and add more features just modifying #***********************************
# collision is needed (features[3]) in Vi， if you want to change it, just delete that
#                # get collision
#                Y[num, 0] += X[num, 3] * rewards_map['Rc']

# this code can be modified to normal VI without function approximation, just use a dictionary to store each value, keep the get features and 
# change get value to return dict[state] and modify update theta by changing dict.valuesto be Y.
class func_approx():
    def __init__(self, my_map, n_agents):
        self.n = len(my_map)
        self.m = len(my_map[0])
        self.my_map = my_map
        self.n_agents = n_agents
        
        self.rate = 1
        self.decay = 0.95

        # get target position, change to a unique code
        for i in range(self.n):
            for j in range(self.m):
                if type(my_map[i][j]) == type('s'):
                    if my_map[i][j] == 'Rs':
                        self.Rs = [i, j]
                        self.Rs_code = i * 1000 + j
                    if my_map[i][j] == 'Rd':
                        self.Rd = [i ,j]
                        self.Rd_code = i * 1000 + j

        # initialize
        #***********************************
        self.theta = np.zeros([7, 1])
        self.theta_old = np.random.rand(*[7, 1])
        #***********************************

    def getFeatures(self, state):
        features = np.zeros_like(self.theta)
        states = eval(state)
        states_code = []

        # change state to unique code and if in negtice state and if in target
        for s in states:
            tmp_code = s[0] * 1000 + s[1]
            states_code.append(tmp_code)
            # in target
            if tmp_code in [self.Rs_code, self.Rd_code]:
                features[0] += 1
            # negative state
            if type(self.my_map[s[0]][s[1]]) == type('s') and self.my_map[s[0]][s[1]] == 'Rw':
                features[1] += 1
            # wall
            if type(self.my_map[s[0]][s[1]]) == type(0) and self.my_map[s[0]][s[1]] == -1:
                features[2] += 1
            #***********************************
            # sum of distance
            features[4] = features[4] + abs(s[0] - self.Rd[0]) + abs(s[1] - self.Rd[1])
            features[5] = features[5] + abs(s[0] - self.Rs[0]) + abs(s[1] - self.Rs[1])
            features[6] = features[6] + (s[0] == 3)
            #************************************
            
        # if collision
        features[3] = len(states_code) - len(set(states_code))
        return features

    def getValues(self, state):
        features = self.getFeatures(state)
        return (self.theta.T.dot(features))[0, 0]

    def updateTheta(self, Data, Y):
        # here Data is N*n_features
        #    Y      is N*1
        self.theta_old = self.theta.copy()
        gradient = (np.linalg.inv(Data.T.dot(Data) + 0.0001 * np.identity(Data.shape[1]))).dot(Data.T).dot(Y)
        self.theta = (1-self.rate)*self.theta_old + self.rate*gradient
        
        self.rate = self.rate * self.decay

In [5]:
# this caolculates the value of next state
moves_map = {0: [0, 0], 1: [-1, 0], 2: [0, 1], 3: [1, 0], 4: [0, -1]}
def getValues(state, acts, my_map, n, m, faFandler):
    next_state = []
    for s, a in zip(eval(state), acts):
        # if in wall
        if type(my_map[s[0]][s[1]]) == type(0) and my_map[s[0]][s[1]] == -1:
            next_state.append(s)
        else:
            # next pos
            movement = moves_map[a]
            next_x = s[0] + movement[0]
            next_y = s[1] + movement[1]

            # if not valid: out the map
            if (next_x < 0 or next_x >= n) or (next_y < 0 or next_y >= m):
                next_x = s[0]
                next_y = s[1]
            # else if target is obstacle
            if type(my_map[next_x][next_y]) == type(0) and my_map[next_x][next_y] == -1:
                next_x = s[0]
                next_y = s[1]

            # next value
            next_state.append([next_x, next_y])

    return faFandler.getValues(str(next_state))

In [6]:
def VI(my_map, pe, discount, horizon, rewards_map, n_agents=2, p1=0.1, p2=0.1):
    # prepare the vlaues
    n = len(my_map)
    m = len(my_map[0])

    # the fa
    faHandler = func_approx(my_map, n_agents)

    # states
    states = getMap(maps={}, needs='v', maxA=n_agents)
    X = np.zeros([len(states), faHandler.theta.shape[0]])
    Y = np.zeros([len(states), 1])

    # actions
    actions = []
    for a, _ in getActionandProb([0] * n_agents, pe):
        actions.append(a)
    tmp_action_value = [0] * len(actions)

    # ite = 0
    ite = 0
    while (np.linalg.norm(faHandler.theta_old - faHandler.theta) >= 1e-2 and ite < horizon) or ite == 0:
        choose = np.random.rand(len(states)) <= p1
        for num, state in enumerate(states.keys()):
            # for selected state:
            if choose[num]:
                # for each action, get its next value
                for i, action in enumerate(actions):
                    tmp_val = 0
                    tmp_p = 0

                    # get next value
                    for acts, p in getActionandProb(action, pe):
                        if np.random.rand() >= p2:
                            pass
                        else:
                            tmp_val += discount * p * getValues(state, acts, my_map, n, m, faHandler)
                            tmp_p += p
                            
                    # calculate value
                    if tmp_p < 1e-5:
                        tmp_val = 0
                    else:
                        tmp_val /= tmp_p

                    # assign
                    tmp_action_value[i] = tmp_val

                # get final assign
                X[num,] = faHandler.getFeatures(state)[:,0]
                Y[num, 0] = max(tmp_action_value)

                # get current reward
                for s in eval(state):
                    Y[num, 0] += rewards_map[my_map[s[0]][s[1]]]

                # get collision
                Y[num, 0] += X[num, 3] * rewards_map['Rc']
            else:
                pass
        # update
        faHandler.updateTheta(X[choose,], Y[choose,])
        ite += 1
        print('iteration {}, result: '.format(ite))
        print(str(faHandler.theta))
    return faHandler

In [7]:
moves_map = {0: [0, 0], 1: [-1, 0], 2: [0, 1], 3: [1, 0], 4: [0, -1]}
my_map = [[0]*5 for _ in range(5)]
my_map[2][0] = 'Rs'
my_map[2][2] = 'Rd'
my_map[1][1] = -1
my_map[2][1] = -1
my_map[1][3] = -1
my_map[2][3] = -1
my_map[4][0] = 'Rw'
my_map[4][1] = 'Rw'
my_map[4][2] = 'Rw'
my_map[4][3] = 'Rw'
my_map[4][4] = 'Rw'

pe = 0.9
discount = 0.9
horizon = np.inf
rewards_map = {'Rd' : 10, 'Rs': 10, 'Rw': -1, 0:0, -1:-10, 'Rc':-5}

n_agents = 2
p1 = 0.3
p2 = 0.8

VI(my_map, pe, discount, horizon, rewards_map, n_agents, p1, p2)

iteration 1, result: 
[[ 9.99995792e+00]
 [-1.00000761e+00]
 [-9.99998214e+00]
 [-4.99990573e+00]
 [ 6.74631685e-07]
 [-8.80389528e-07]
 [ 2.29104383e-06]]
iteration 2, result: 
[[ 17.76728958]
 [ -1.44046085]
 [-19.01414694]
 [ -7.38126481]
 [  0.56122913]
 [ -0.17282772]
 [  2.51494829]]
iteration 3, result: 
[[ 24.52194258]
 [ -1.96659153]
 [-26.82069846]
 [ -0.99801902]
 [  1.10646445]
 [ -0.12709801]
 [  4.959999  ]]
iteration 4, result: 
[[ 29.61462598]
 [ -0.8853414 ]
 [-32.58057107]
 [ -0.53575209]
 [  2.05479185]
 [ -0.30660078]
 [  8.79742913]]
iteration 5, result: 
[[ 35.26992113]
 [  0.75177244]
 [-39.14388753]
 [ -6.7820837 ]
 [  3.26504714]
 [ -0.50461218]
 [ 11.70433947]]
iteration 6, result: 
[[ 38.65920226]
 [  0.86267817]
 [-44.98761656]
 [ -3.88092877]
 [  5.05960638]
 [ -0.78634979]
 [ 12.9094344 ]]
iteration 7, result: 
[[ 41.25835878]
 [ -2.73282557]
 [-53.24203062]
 [ -4.24152619]
 [  6.23230789]
 [ -0.09948268]
 [ 13.65713654]]
iteration 8, result: 
[[ 4.3609865

iteration 59, result: 
[[  60.60222687]
 [ -14.20776279]
 [-126.53246662]
 [  -9.51460177]
 [  10.58789891]
 [  14.94753153]
 [  18.41656571]]
iteration 60, result: 
[[  60.71890285]
 [ -14.30946781]
 [-126.74139907]
 [  -9.78229939]
 [  10.57126821]
 [  15.02094495]
 [  18.35878187]]
iteration 61, result: 
[[  60.82255182]
 [ -14.41469935]
 [-127.01987226]
 [  -9.29107944]
 [  10.51169918]
 [  15.14087251]
 [  18.28167143]]
iteration 62, result: 
[[  60.83758157]
 [ -14.44742488]
 [-127.24680019]
 [  -9.23760848]
 [  10.49755097]
 [  15.20490706]
 [  18.36709208]]
iteration 63, result: 
[[  60.90535817]
 [ -14.36714367]
 [-127.36883248]
 [  -9.51637464]
 [  10.49842112]
 [  15.24414633]
 [  18.38278041]]
iteration 64, result: 
[[  60.98584763]
 [ -14.61611496]
 [-127.49275303]
 [  -9.93380272]
 [  10.51091683]
 [  15.3059999 ]
 [  18.24116983]]
iteration 65, result: 
[[  60.95522533]
 [ -14.72064481]
 [-127.65406865]
 [  -9.76430707]
 [  10.52476904]
 [  15.33981376]
 [  18.28206989]]